# Load Levels

In [30]:
levels = [open(f"levels/level{i+1}.txt", "r").readline().strip() for i in range(10)]

# Customize Game Class

In [31]:
from game import Game

RIGHT_DIRECT_MOVE = "0"
RIGHT_JUMP__MOVE = "1"
RIGHT_DODGE_MOVE = "2"

BLANKK = "_"
GOOMPA = "G"
LAKIPO = "L"
MUSHRM = "M"


class MyGame(Game):
    # modified score function
    def get_score(
        self,
        actions,
        step_score=1,
        honor_score=1,
        bonus_score=2,
        enemy_encounter_score=2,
        win_score=5,
        lose_penalty=0,
        jump_penalty=0.2,
        is_lose_seq_continue_accepted=True,
    ):
        current_level = self.levels[self.current_level_index]
        won = True
        score = 0  # previously named step
        for step in range(self.current_level_len - 1):
            current_step = current_level[step]
            if current_step == BLANKK and (
                step == 0 or actions[step - 1] != RIGHT_JUMP__MOVE
            ):
                score += step_score
            elif current_step == BLANKK and actions[step - 1] == RIGHT_JUMP__MOVE:
                score += step_score - jump_penalty
            # bonus encounter state
            elif current_step == MUSHRM and actions[step - 1] != RIGHT_JUMP__MOVE:
                score += bonus_score
            elif current_step == MUSHRM and actions[step - 1] == RIGHT_JUMP__MOVE:
                score += step_score
            # enemy encounter state
            elif current_step == GOOMPA and actions[step - 2] == RIGHT_JUMP__MOVE:
                score += step_score + enemy_encounter_score
            elif current_step == GOOMPA and actions[step - 1] == RIGHT_JUMP__MOVE:
                score += step_score
            elif current_step == LAKIPO and actions[step - 1] == RIGHT_DODGE_MOVE:
                score += step_score
            # lose state
            else:
                won = False
                score -= lose_penalty
                if is_lose_seq_continue_accepted:
                    continue
                else:
                    break
        # honorable win
        if won and actions[len(current_level) - 1] == RIGHT_JUMP__MOVE:
            score += honor_score

        return won, score + (win_score if won else 0)

    # bulk running utilities
    def get_score_for_next(self, move):
        self.load_next_level()
        return self.get_score(move)

    def get_score_for_each(self, move_list):
        self.current_level_index = -1
        self.current_level_len = 0
        scores = [self.get_score_for_next(move) for move in move_list]
        return scores

# Initialization

In [32]:
import random
import numpy as np


def generate_random_string(length, possible_actions="012"):
    """randomly generates a sequence of given possible_actions

    default actions express equal chance of presence for each action in random solution

    e.g. you can pass "0012" for double chance of direct move presence in random solution
    """
    return "".join(random.choice(possible_actions) for i in range(length))


INIT_POPULATION_SIZE = 200
population = np.array(
    [
        [generate_random_string(len(l)) for l in levels]
        for i in range(INIT_POPULATION_SIZE)
    ]
)

# Selection

In [33]:
import math

def round_wheel_selection(inputs, scores):
    selected_inputs = []
    selected_scores = []
    points = [random.randint(0, int(sum(scores) - 1)) for i in range(math.ceil(len(inputs) / 2))]
    progressive_sum = 0
    for i in range(len(inputs)):
        for point in points:
            if point >= progressive_sum and point < progressive_sum + scores[i]:
                selected_inputs.append(inputs[i])
                selected_scores.append(scores[i])
        progressive_sum += scores[i]
    return selected_inputs, selected_scores


def fitness_based_selection(inputs, scores):
    sorted_inputs, sorted_scores = zip(
        *sorted(list(zip(inputs, scores)), key=lambda x: x[1])
    )
    return sorted_inputs[len(inputs) // 2 :], sorted_scores[len(sorted_scores) // 2 :]


# test
print("round_wheel_selection", round_wheel_selection(["111", "222", "000"], [1, 2, 3]))
print("fitness_based_selection", fitness_based_selection(["111", "222", "000"], [1, 2, 3]))

round_wheel_selection (['000', '000'], [3, 3])
fitness_based_selection (('222', '000'), (2, 3))


# Recombination

In [34]:
def crossover(parents, points):
    points = sorted(points)
    # the last point is a helper expressing the end of chromosome(not last gene!)
    points.append(len(parents[0])) if points[-1] != len(parents[0]) else None
    offsprings = list(parents)
    last_point_ind = 0
    for round in range(len(points)):
        # if points be more that parents
        start_parent_ind = round % len(parents)

        for child_ind in range(len(offsprings)):
            parent_ind = (child_ind + start_parent_ind) % len(parents)
            offsprings[child_ind] = (
                offsprings[child_ind][:last_point_ind]
                + parents[parent_ind][last_point_ind : points[round]]
                + parents[child_ind][points[round] :]
            )
        last_point_ind = points[round]
    return offsprings

# test
print("one point", crossover(["111", "222", "000"],[1]))
print("two point", crossover(["111", "222", "000"],[1,2]))

one point ['122', '200', '011']
two point ['120', '201', '012']


# Mutation

In [35]:
def mutate_each_gene(chrom_lst, mut_prob=0.2, actions="00012"):
    for chrom_ind in range(len(chrom_lst)):
        for gene_ind in range(len(chrom_lst[chrom_ind])):
            if random.random() <= mut_prob:
                chrom_lst[chrom_ind] = (
                    chrom_lst[chrom_ind][:gene_ind]
                    + random.choice(actions)
                    + chrom_lst[chrom_ind][gene_ind + 1 :]
                )
    return chrom_lst


def mutate_random_gene(chrom_lst, mut_prob=0.2, actions="00012", try_times=1):
    for chrom_ind in range(len(chrom_lst)):
        if random.random() <= mut_prob:
            for try_time in range(try_times):
                gene_ind = random.randint(0, chrom_ind - 1) if chrom_ind != 0 else 0
                chrom_lst[chrom_ind] = (
                    chrom_lst[chrom_ind][:gene_ind]
                    + random.choice(actions)
                    + chrom_lst[chrom_ind][gene_ind + 1 :]
                )
    return chrom_lst


# test
print("mutate_each_gene", mutate_each_gene(["111", "222"]))
print("mutate_random_gene", mutate_random_gene(["111", "222"]))

mutate_each_gene ['111', '202']
mutate_random_gene ['011', '222']


# Stop Criterion

In [36]:
EPSILON = 0.1
MAX_GENRATION = 10
def check_stop_criterion(generation_number, mean_dif):
    return generation_number >= 10 or mean_dif <= 0.1
    

In [ ]:
import numpy as np

g = MyGame(levels)


score_mean = []
gn = 0
while not check_stop_criterion(gn, score_mean if gn > 1 else 999999):
    # bulk running
    scores = np.array([g.get_score_for_each(pop) for pop in population])
    # selection
    sel_pop, sel_scores = round_wheel_selection(population[:, 8], scores[:, 8, 1])
    # recombin
    off_springs = crossover(sel_pop, [len(sel_pop[0]) // 2])
    # mutation
    mutated_offsprings = mutate_each_gene(off_springs)
    # mutated_offsprings = mutate_randome_gene(off_springs)
    # next gen
    present_parents, _ = fitness_based_selection(population[:, 8], scores[:, 8, 1])
    population[:, 8] = off_springs + list(present_parents)
    gn += 1
    score_mean.append(
        np.mean(
            [[result[1] for result in sample] for sample in scores], axis=0
        ).reshape(-1, 1)
    )

# Diagrams

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

score_changes = np.array([score_mean]*10)

fig, ax = plt.subplots(4, 3, figsize=(15, 20))

ln = 1
for j in range(4):
    for i in range(3):
        ax[j, i].plot(score_changes[:, ln - 1])
        ax[j, i].set_title(f"level {ln}")
        ln += 1
        if ln > 10: break
plt.show()